In [3]:
import json
import gensim
from nltk.corpus import stopwords
import nltk
import re
import numpy as np
from nltk.stem import PorterStemmer
import os
import enchant
from nltk.tokenize import RegexpTokenizer

In [2]:
#fetch all reviews
data=[]
for root, dirs, files in os.walk('../Reviews/'):
    for name in dirs:                             #all review folders
                with open('../Reviews/'+name+'/review.txt') as f:
            data=data+json.load(f)
            f.close()
dataset=[]
dataset=np.asarray(data)
dataset=np.concatenate(dataset)               #contains all the reviews

In [3]:
#initialising dictionary, tokenizer, stemmer
dictionary = enchant.Dict("en_US")
tokenizer=RegexpTokenizer('[a-zA-Z]+')
ps=PorterStemmer()
all_stopwords=stopwords.words('english')+['ibm','amazon']

In [8]:
def preprocess(sentence):
    dataset=[]
    #print 'sentence : '+sentence
    temp=tokenizer.tokenize(sentence)
    #print 'tokenizer : '+str(temp)
    only_english=[word for word in temp if dictionary.check(word)== True]  # remove non-english words
    #print 'removed all non-english words : '+str(only_english)
    return  only_english           # returns tokens

In [7]:
def tokenize_review(review):                # for tokenizing single reviews
    review=review.lower()
    review_tokens=[]
    review=review.replace('.',',')        
    #print 'Review : '+review
    sentences=re.split(' and | but |, ',review) 
    for sentence in sentences:
        #print ' '
        temp=preprocess(sentence)
        if(len(temp)>1):
            review_tokens.append(temp)
    return review_tokens

In [53]:
#only for displaying
tokenize_review(dataset[77])

Review :  yes,it's a good company to work on and learn new things in your track,lot of opportunities provided by the company to learn cross module experience, it also has a good work life balance 
 
sentence :  yes,it's a good company to work on
tokenizer : [u'yes', u'it', u's', u'a', u'good', u'company', u'to', u'work', u'on']
removed all non-english words : [u'yes', u'it', u's', u'a', u'good', u'company', u'to', u'work', u'on']
 
sentence : learn new things in your track,lot of opportunities provided by the company to learn cross module experience
tokenizer : [u'learn', u'new', u'things', u'in', u'your', u'track', u'lot', u'of', u'opportunities', u'provided', u'by', u'the', u'company', u'to', u'learn', u'cross', u'module', u'experience']
removed all non-english words : [u'learn', u'new', u'things', u'in', u'your', u'track', u'lot', u'of', u'opportunities', u'provided', u'by', u'the', u'company', u'to', u'learn', u'cross', u'module', u'experience']
 
sentence : it also has a good work

[[u'yes', u'it', u's', u'a', u'good', u'company', u'to', u'work', u'on'],
 [u'learn',
  u'new',
  u'things',
  u'in',
  u'your',
  u'track',
  u'lot',
  u'of',
  u'opportunities',
  u'provided',
  u'by',
  u'the',
  u'company',
  u'to',
  u'learn',
  u'cross',
  u'module',
  u'experience'],
 [u'it', u'also', u'has', u'a', u'good', u'work', u'life', u'balance']]

In [9]:
def tokenize_all(dataset):               # for tokenizing all the reviews
    tokens=[]
    for review in dataset:
        tokens=tokens+tokenize_review(review)
    return tokens    

In [10]:
#'tokens' contain all the tokens
tokens=tokenize_all(dataset)

In [11]:
#storing entire tokens to a a file
with open('tokens.txt','w') as f:
    json.dump(tokens,f)     

In [12]:
len(tokens)

475332

In [13]:
# training word2vec
model = gensim.models.Word2Vec(tokens, min_count=5, workers=2)
model.save('model')              #saves the model

In [10]:
#testing most_similar
model.most_similar('enjoy')

[(u'enjoyed', 0.8321654796600342),
 (u'love', 0.8068959712982178),
 (u'loved', 0.800926148891449),
 (u'liked', 0.7872418165206909),
 (u'hated', 0.6736726760864258),
 (u'appreciate', 0.6420800685882568),
 (u'miss', 0.6189007759094238),
 (u'hate', 0.6054211854934692),
 (u'enjoying', 0.5799765586853027),
 (u'believe', 0.5136904716491699)]

In [65]:
#testing similarity
#model.similarity('job','work')

In [8]:
# testing load feature
#model=gensim.models.Word2Vec.load('model')
#new_model.most_similar('work')